# Fake data from Bayesian networks


In [ ]:
import numpy as np
from fake_data_for_learning import BayesianNodeRV as BNRV
from fake_data_for_learning import FakeDataBayesianNetwork as FDBN

%matplotlib inline

## Example
$X = (X0, X1)$, where $X$ admits the graph

G: X0 -> X1

First define random node variables for $X0$ and $X1 | X0$, where $X0$ can be sampled with no additional arguments, while $X1 | X0$ must be provided a value for $X0$ to generate a random sample.

### Sampling from the nodes

In [ ]:
pt_X0 = np.array([0., 1])
rv0 = BNRV('X0', pt_X0)
x0 = rv0.rvs()
x0

In [ ]:
pt_X1cX0 = np.array([
        [0.2, 0.8],
        [0.7, 0.3],
    ])

rv1c0 = BNRV('X1', pt_X1cX0, parent_names=['X0'])
parent_values={'X0': 1}
# Get probability table when X0=1
rv1c0.get_pt(parent_values)

In [ ]:
#def test_rvs_1c0(self):
draw = rv1c0.rvs(size=10, parent_values={'X0': 1})
draw

### Sampling from the graph

To generate a random sample for $X$, we define the adjacency graph of $G$ and use this to first sample from the Eve-nodes (no parents), and then Eve's immediate descendents, and so on. 

In [ ]:
bn = FDBN(rv0, rv1c0)

In [ ]:
# Visualize the graph
bn.draw_graph()

In [ ]:
# Sample
bn.rvs()

## Example

$X = (X0, X1, X2)$, where $X$ admits the graph

G: X0 -> X2 <- X1

In [ ]:
# X1
pt_X1 = np.array([0.5, 0.3, 0.2])
rv1 = BNRV('X1', pt_X1)

# X2 | X0, X1
pt_X2cX0X1 = np.array([
    [
        [0., 1.0],
        [0.2, 0.8],
        [0.1, 0.9]
    ],
    [
        [0.5, 0.5],
        [0.3, 0.7],
        [0.9, 0.1]
    ],

])

rv2c01 = BNRV('X2', pt_X2cX0X1, parent_names=['X0', 'X1'])

bn2c01 = FDBN(rv0, rv1, rv2c01)
bn2c01.draw_graph()

In [ ]:
bn2c01.rvs()